In [1]:
#Q9
import pandas as pd
import numpy as np

def calculate_distance_matrix(csv_file):
    df = pd.read_csv(csv_file)
    
    required_columns = {'id_start', 'id_end', 'distance'}
    if not required_columns.issubset(df.columns):
        raise ValueError("CSV must contain 'id_start', 'id_end', and 'distance' columns")

    locations = set(df['id_start']).union(set(df['id_end']))
    locations = list(locations)
    
    distance_matrix = pd.DataFrame(index=locations, columns=locations)
    distance_matrix = distance_matrix.fillna(np.inf)

    for index, row in df.iterrows():
        loc_a = row['id_start']
        loc_b = row['id_end']
        distance = row['distance']
        
        distance_matrix.loc[loc_a, loc_b] = distance
        distance_matrix.loc[loc_b, loc_a] = distance

    np.fill_diagonal(distance_matrix.values, 0)

    for k in locations:
        for i in locations:
            for j in locations:
                if distance_matrix.loc[i, k] + distance_matrix.loc[k, j] < distance_matrix.loc[i, j]:
                    distance_matrix.loc[i, j] = distance_matrix.loc[i, k] + distance_matrix.loc[k, j]
                    
    distance_matrix.replace(np.inf, np.nan, inplace=True)

    print("Distance Matrix:")
    print(distance_matrix)

    return distance_matrix

distance_df = calculate_distance_matrix('dataset-2.csv')
print("Returned Distance DataFrame:")
print(distance_df)

Distance Matrix:
         1001472  1001488  1004354  1004355  1004356  1001400  1001402  \
1001472      0.0    179.8    173.8    171.8    175.8    444.3    434.6   
1001488    179.8      0.0      6.0      8.0      4.0    264.5    254.8   
1004354    173.8      6.0      0.0      2.0      2.0    270.5    260.8   
1004355    171.8      8.0      2.0      0.0      4.0    272.5    262.8   
1004356    175.8      4.0      2.0      4.0      0.0    268.5    258.8   
1001400    444.3    264.5    270.5    272.5    268.5      0.0      9.7   
1001402    434.6    254.8    260.8    262.8    258.8      9.7      0.0   
1001404    414.4    234.6    240.6    242.6    238.6     29.9     20.2   
1001406    398.4    218.6    224.6    226.6    222.6     45.9     36.2   
1001408    376.7    196.9    202.9    204.9    200.9     67.6     57.9   
1001410    365.6    185.8    191.8    193.8    189.8     78.7     69.0   
1001412    350.0    170.2    176.2    178.2    174.2     94.3     84.6   
1001414    331.8    1

In [2]:
#Q10
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    unrolled_data = []

    locations = distance_matrix.index

    for i in locations:
        for j in locations:
            if i != j:
                distance = distance_matrix.loc[i, j]
                if not pd.isna(distance):
                    unrolled_data.append({'id_start': i, 'id_end': j, 'distance': distance})

    unrolled_df = pd.DataFrame(unrolled_data)

    return unrolled_df

unrolled_df = unroll_distance_matrix(distance_df)
print("Unrolled Distance DataFrame:")
print(unrolled_df)


Unrolled Distance DataFrame:
      id_start   id_end  distance
0      1001472  1001488     179.8
1      1001472  1004354     173.8
2      1001472  1004355     171.8
3      1001472  1004356     175.8
4      1001472  1001400     444.3
...        ...      ...       ...
1801   1001470  1001461      61.6
1802   1001470  1001462      56.5
1803   1001470  1001464      29.8
1804   1001470  1001466      21.3
1805   1001470  1001468      10.6

[1806 rows x 3 columns]


In [3]:
#Q11
def find_ids_within_ten_percentage_threshold(df, reference_value):
    related_distances = df[df['id_start'] == reference_value]['distance']

    if related_distances.empty:
        return []

    average_distance = related_distances.mean()

    lower_bound = average_distance * 0.9
    upper_bound = average_distance * 1.1
 
    ids_within_threshold = df[(df['distance'] >= lower_bound) & (df['distance'] <= upper_bound)]
   
    return sorted(ids_within_threshold['id_start'].tolist())

reference_value = unrolled_df['id_start']

ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_df, reference_value)

print("IDs within 10% of average distance for reference ID {}: {}".format(reference_value, ids_within_threshold))


IDs within 10% of average distance for reference ID 0       1001472
1       1001472
2       1001472
3       1001472
4       1001472
         ...   
1801    1001470
1802    1001470
1803    1001470
1804    1001470
1805    1001470
Name: id_start, Length: 1806, dtype: int64: [1001400, 1001400, 1001402, 1001402, 1001404, 1001404, 1001406, 1001406, 1001408, 1001408, 1001408, 1001410, 1001410, 1001410, 1001412, 1001412, 1001412, 1001412, 1001414, 1001414, 1001414, 1001414, 1001414, 1001416, 1001416, 1001416, 1001416, 1001416, 1001416, 1001416, 1001416, 1001418, 1001418, 1001418, 1001418, 1001418, 1001418, 1001418, 1001418, 1001420, 1001420, 1001420, 1001422, 1001422, 1001422, 1001424, 1001424, 1001424, 1001424, 1001426, 1001426, 1001426, 1001426, 1001426, 1001428, 1001428, 1001428, 1001428, 1001428, 1001430, 1001430, 1001430, 1001430, 1001430, 1001432, 1001432, 1001432, 1001432, 1001432, 1001432, 1001434, 1001434, 1001434, 1001436, 1001436, 1001436, 1001436, 1001437, 1001437, 1001437, 1001437

In [4]:
#Q12
import pandas as pd

def calculate_toll_rate(df):
    rates = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }

    for vehicle, rate in rates.items():
        df[vehicle] = df['distance'] * rate

    return df

result_df = calculate_toll_rate(unrolled_df)
print(result_df)


      id_start   id_end  distance    moto     car      rv     bus    truck
0      1001472  1001488     179.8  143.84  215.76  269.70  395.56   647.28
1      1001472  1004354     173.8  139.04  208.56  260.70  382.36   625.68
2      1001472  1004355     171.8  137.44  206.16  257.70  377.96   618.48
3      1001472  1004356     175.8  140.64  210.96  263.70  386.76   632.88
4      1001472  1001400     444.3  355.44  533.16  666.45  977.46  1599.48
...        ...      ...       ...     ...     ...     ...     ...      ...
1801   1001470  1001461      61.6   49.28   73.92   92.40  135.52   221.76
1802   1001470  1001462      56.5   45.20   67.80   84.75  124.30   203.40
1803   1001470  1001464      29.8   23.84   35.76   44.70   65.56   107.28
1804   1001470  1001466      21.3   17.04   25.56   31.95   46.86    76.68
1805   1001470  1001468      10.6    8.48   12.72   15.90   23.32    38.16

[1806 rows x 8 columns]


In [5]:
#Q13
import pandas as pd
from datetime import time

def calculate_time_based_toll_rates(df):
    time_ranges = {
        'weekdays': [
            (time(0, 0), time(10, 0), 0.8),
            (time(10, 0), time(18, 0), 1.2),
            (time(18, 0), time(23, 59, 59), 0.8)
        ],
        'weekends': [
            (time(0, 0), time(23, 59, 59), 0.7)
        ]
    }

    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    records = []

    for day in days:
        if day in ['Saturday', 'Sunday']:
            discount = time_ranges['weekends'][0][2] 
        else:
            discount_factors = []
            for start, end, factor in time_ranges['weekdays']:
                discount_factors.append((start, end, factor))

        for hour in range(24):
            start_time = time(hour, 0)
            end_time = time(hour, 59, 59)

            if day not in ['Saturday', 'Sunday']:
                for start, end, factor in discount_factors:
                    if start_time >= start and end_time <= end:
                        discount = factor
                        break

            for _, row in df.iterrows():
                discounted_row = {
                    'start_day': day,
                    'start_time': start_time,
                    'end_day': day,
                    'end_time': end_time,
                    'id_start': row['id_start'],
                    'id_end': row['id_end'],
                    'moto': row['moto'] * discount,
                    'car': row['car'] * discount,
                    'rv': row['rv'] * discount,
                    'bus': row['bus'] * discount,
                    'truck': row['truck'] * discount
                }
                records.append(discounted_row)

    result_df = pd.DataFrame(records)

    return result_df

results_df = calculate_time_based_toll_rates(result_df)
print(results_df)


       start_day start_time end_day  end_time   id_start     id_end     moto  \
0         Monday   00:00:00  Monday  00:59:59  1001472.0  1001488.0  115.072   
1         Monday   00:00:00  Monday  00:59:59  1001472.0  1004354.0  111.232   
2         Monday   00:00:00  Monday  00:59:59  1001472.0  1004355.0  109.952   
3         Monday   00:00:00  Monday  00:59:59  1001472.0  1004356.0  112.512   
4         Monday   00:00:00  Monday  00:59:59  1001472.0  1001400.0  284.352   
...          ...        ...     ...       ...        ...        ...      ...   
303403    Sunday   23:00:00  Sunday  23:59:59  1001470.0  1001461.0   34.496   
303404    Sunday   23:00:00  Sunday  23:59:59  1001470.0  1001462.0   31.640   
303405    Sunday   23:00:00  Sunday  23:59:59  1001470.0  1001464.0   16.688   
303406    Sunday   23:00:00  Sunday  23:59:59  1001470.0  1001466.0   11.928   
303407    Sunday   23:00:00  Sunday  23:59:59  1001470.0  1001468.0    5.936   

            car       rv      bus     t